<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/importing-network-from-table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Network From Table


## Yihang Xin and Alex Pico
## 2025-01-13

In addition to importing networks in network file formats, such as sif and xgmml, Cytoscape also supports importing networks from tabular data. In this notebook, the data table represents protein-protein interaction data from a mass-spectrometry experiment.



# Installation
The following chunk of code installs the `py4cytoscape` module.

In [1]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)



In [2]:
#_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.11.0' # optional
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


Loading Javascript client ... f01ea9f5-9a3b-40e7-8aad-24052cc9adce on https://jupyter-bridge.cytoscape.org


<IPython.core.display.Javascript object>

# Prerequisites
In addition to this package (py4cytoscape version 1.11.0), you will need:

* Latest version of Cytoscape, which can be downloaded from https://cytoscape.org/download.html. Simply follow the installation instructions on screen.

* Complete installation wizard

* Launch Cytoscape

You can also install app inside Python notebook by running "py4cytoscape.install_app('Your App')"

# Import the required package¶


In [3]:
import os
import sys
import pandas as pd
import py4cytoscape as p4c

# Setup Cytoscape


In [4]:
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'cytoscapeVersion': '3.9.0',
 'automationAPIVersion': '1.2.0',
 'py4cytoscapeVersion': '0.0.10'}

# Background
The data used for this protocol represents interactions between human and HIV proteins by Jäger et al (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3310911/). In this quantitative AP-MS experiment, a relatively small number of bait proteins were used to pull down a larger set of prey proteins.



# Import Network


First we need to read in the example data file:



In [5]:
apms_data = pd.read_csv("https://raw.githubusercontent.com/cytoscape/cytoscape-automation/master/for-scripters/R/notebooks/AP-MS/ap-ms-demodata_simple.csv")

In [6]:
apms_data.head()

,Bait,Prey,HEKScore,AP-MS Score
0,GAG,THRAP3,0.807,0.563
1,GAG,SEPSECS,0.814,0.507
2,GAG,IVNS1ABP,0.753,0.506
3,GAG,DDX49,0.824,0.412
4,GAG,PRMT1,0.758,0.397


Now we can create a data frame for the network edges (interactions) using the imported data. We can also add the AP-MS score from the data as an edge attribute:

In [7]:
edge_data = {'source':apms_data["Bait"],
             'target':apms_data["Prey"],
             'AP-MS Score':apms_data["AP-MS Score"]
            }
edges = pd.DataFrame(data=edge_data, columns=['source', 'target','AP-MS Score'])
edges.head()

,source,target,AP-MS Score
0,GAG,THRAP3,0.563
1,GAG,SEPSECS,0.507
2,GAG,IVNS1ABP,0.506
3,GAG,DDX49,0.412
4,GAG,PRMT1,0.397


Finally, we use the edge data fram to create the network. Note that we don’t need to define a data frame for nodes, as all nodes in this case are represented in the edge data frame.



In [8]:
p4c.create_network_from_data_frames(edges=edges, title='apms network', collection="apms collection")

Applying default style...
Applying preferred layout


{'networkSUID': 57053}

The imported network consists of multiple smaller subnetworks, each representing a bait node and its associated prey nodes.



# Loading Data


There is one other column of data for the “Prey” proteins that we want to load into this network, the “HEKScore”.

In this data, the Prey nodes are repeated for each interactions with a Bait node, so the data contains different values for the same attribute (for example HEKScore), for each Prey node. During import, the last value imported will overwrite prior values and visualizations using this attribute thus only shows the last value.

In [9]:
p4c.load_table_data(apms_data[["Prey","HEKScore"]], data_key_column='Prey')

'Success: Data loaded in defaultnode table'